<a href="https://colab.research.google.com/github/jarrydmartinx/deep-rl/blob/master/implicit-quantile-networks-theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Implicit Quantile Networks (IQN) (Dabney, 2018)

###Summary
* Uses quantile regression to approximate the full quantile function for the state-action return distribution
* It's best viewed as a simple distributional generalisation of DQN
* Despite forgoing any other improvements, IQN significantly outperforms QR-DQN and nearly matches the performance of Rainbow, which combines many orthogonal advances. Even improves over Rainbow on the hardest Atari games

### Viewing IQN as an extension of DQN
* Empirically, distributional RL algorithms show improved sample complexity and final performance, as well as increased robustness to hyperparameter variation (Barth-Maron, 2018)
* In distributional RL, the distribution over returns (the law of $Z^\pi$) is considered instead of the scalar value function $Q^\pi$ that is its expectation.


###Distributional Bellman Equation and Optimality Operator $\mathcal{T}$
The Distributional Bellman equation
$$ Z\pi(x,a) :\overset{D}{=} R(x,a) + \gamma Z^\pi(X',A')$$
The Distributional Bellman optimality operator
$$\mathcal{T} Z(x,a) :\overset{D}{=} R(x,a) + \gamma Z(X', \arg\max_{a'\in\mathcal{A}} \mathop{\mathbb{E}}Z(X', a') \qquad X' \sim P(\cdot\mid x,a) $$
#### Properties of $\mathcal{T}$
* While the distributional Bellman operator for policy evaluation is a contraction in the $p$-Wasserstein distance (Bellemare, 2017), this no longer holds for the control case. 
* Convergence to the optimal policy can be established but requires a more complicated argument

### The $p$-Wasserstein Metric and Quantile Functions

>The $p$-Wasserstein metric, for $p\in[1,\infty]$ has been a topic of increasing interest in generative modeling, because unlike the KL divergence, the **Wasserstein metric inherently trades off approximate solutions with likelihoods.**

It plays a key role in recent results in distributional RL (Bellemare et al.. 2017; Dabney et al., 2018). Note: *c.d.f's* $\equiv$ *quantile functions*
>The $p$-Wasserstein distance is the **$L_p$ metric on inverse cumulative distribution functions (c.d.f.), also known as quantile functions**
#### The class of optimal transport metrics
 * Members of this class express distances between distributions in terms of the minimal cost for transporting mass to make the two distributions identical (c.f. Earth-Mover metaphor)
 * This cost is given in terms of some metric $c: \mathcal{X} \times \mathcal{X} \to \mathbb{R}^{\geq 0}$ on the underlying space $\mathcal{X}$. As mentioned, the $p$-Wasserstein metric corresponds to $c = L_p$.
* The Wasserstein metrics are particularly interesting for RL due to the predominant use of $L_p$ spaces in mean-value reinforcment learning (normal Sutton RL).

>Rowland (2018) showed that the Categoritcal algorithms are a contraction in the Cramer distance, which is the $L_2$ metric on c.d.f's (quantile functions)

### Review of C51 (Categorical DQN) and QR-DQN
####C51:
* **Model**: parameterize the return distribution as a categorical distribution over a fixed set of equidistant points
* **Loss**: minimize the KL divergence to the projected distributional Bellman target.
* **Tragedy**: Theory/practice gap - the algorithm doesn't actually minimize the Wasserstein distance. 
* NOTE: **Rainbow** is just C51 + prioritized experience replay + $n$-step updates + the dueling architecture. Still the current SOA in Atari-57
#### QR-DQN
* Proposes the use of quantile regression for distributional RL and showed that: 
> By choosing the quantile targets suitably the resulting projected distributional Bellman operator is a contraction in the $\infty$-Wasserstein metric.

* Algorithm: By estimating the quantile function at precisely chosen points, QR-DQN minimizes the Wasserstein distance to the distributional Bellman target.

## Implicit Quantile Networks
### Model
* IQN is a deterministic parametric function trained to reparameterize samples from a *base distribution*, e.g. $\tau\in U([0,1])$, to the respective quantile values of a target distribution.
* This is an effective way to learn an implicit representation of the return distribution, yielding a powerful function approximator for a new DQN-like agent.
* As above, $F_Z^{-1}(\tau)$ is the quantile function at $\tau\in[0,1]$ for the random variable $Z$.
* For notational simplicity we write $ Z_{\tau} := F_z^{-1}$, thus for $\tau\sim U([0,1])$ the resulting state-action return distribution sample is $Z_{\tau}(x,a) \sim Z(x,a)$

### Loss
For two samples $\tau,\tau'\sim U([0,1])$, and policy $\pi_\beta$, the sampled temporal difference error (TD) error.  

### Aside: Risk sensitive RL
* To do risk sensitive RL they define a *distortion risk measure*, with identity corresponding to risk-neutrality (which is all we care about)
* Any reference to a function $\beta: [0,1]\to[0,1]$ in the algorithm is only important for this, so I leave it out.

